In [1]:
from flask import Flask, render_template, request, redirect, url_for, flash
import subprocess, os, ast, ffmpeg
from pytubefix import YouTube
import assemblyai as aai
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from pydub import AudioSegment
from datetime import timedelta
from moviepy import *
from content_aware_crop import load_yolov8_model, process_video, add_audio_to_video

In [22]:
def clear_video_folder():
    video_folder = 'videos/'
    for folder in [video_folder, 'clips']:
        for filename in os.listdir(folder):
            file_path = os.path.join(folder, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)
            
def download_yt(yt):
    save_folder = 'videos'

    video_stream = yt.streams.filter(adaptive=True, file_extension='mp4', only_video=True).order_by('resolution').desc().first()
    audio_stream = yt.streams.filter(adaptive=True, file_extension='mp4', only_audio=True).order_by('abr').desc().first()

    video_path = os.path.join(save_folder, 'videostream.mp4')
    audio_path = os.path.join(save_folder, 'audiostream.mp4')
    file = os.path.join(save_folder, 'input_video.mp4')

    #I can only get the highest quality video by downloading video and audio separately (then combining them after)
    video_stream.download(output_path=save_folder, filename='videostream.mp4')
    audio_stream.download(output_path=save_folder, filename='audiostream.mp4')

    subprocess.run(f"ffmpeg -i {video_path} -i {audio_path} -c copy {file}")

def get_video_length(video):
    result = subprocess.run(["ffprobe", "-v", "error", "-show_entries",
                            "format=duration", "-of",
                            "default=noprint_wrappers=1:nokey=1", video],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    return float(result.stdout)

def transcribe_video():
    #Get compressed audio file only for shorter file processing
    sound = AudioSegment.from_file("videos\\input_video.mp4")
    sound.export("videos\\transcribe.mp3", format="mp3", bitrate="64k")
    
    aai.settings.api_key = os.environ["AAI_TOKEN"]
    transcriber = aai.Transcriber()
    transcript = transcriber.transcribe(f"videos\\transcribe.mp3")
    sentences = transcript.get_sentences()
    
    #format output
    output = []
    for sentence in sentences: 
        output.append(f'{sentence.text} {sentence.start}-{sentence.end}')
    return ('\n'.join(output))

def choose_sections(clip_length, num_clips, transcript):
    endpoint = "https://models.github.ai/inference"
    model = "openai/gpt-4.1"
    token = os.environ["GITHUB_TOKEN"]

    with open('prompt.txt', 'r') as f:
        prompt = f.read()

    client = ChatCompletionsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token),
    )

    response = client.complete(
        messages=[
            SystemMessage(prompt),
            UserMessage(f"""{num_clips} clips of {clip_length} seconds
                        {transcript}"""),
        ],
        temperature=1,
        top_p=1,
        model=model
    )

    return(response.choices[0].message.content)

def cut_video(sections):
    def segment(inputfile, start_time, end_time, outputfile): 
        cmd = [
            "bin\\ffmpeg.exe",
            "-ss", str(timedelta(milliseconds=start_time)),
            "-to", str(timedelta(milliseconds=end_time)),
            "-i", inputfile,
            "-vf", "scale=-2:1920",
            "-c:v", "libx264",
            "-c:a", "aac",
            outputfile
        ]
        subprocess.run(cmd)
    
    
    
    video_file = "videos\input_video.mp4"
    i = 1
    for start, end in sections:
        temp_file = f"videos\\trimmed\\clip_{i}.mp4"
        segment(video_file, start, end, temp_file)
        i += 1
        
def crop_clips():
    folder = "videos\\trimmed"
    i = 1
    for clip in os.listdir(folder):
        input_video_path = os.path.join(folder, clip)
        processed_video_path = f"videos\\cropped\\clip_{i}_noaudio.mp4"
        final_output_path = f"videos\\cropped\\clip_{i}.mp4"
        yolo_model_path = "static\\assets\\yolov8n.pt"
        
        model = load_yolov8_model(yolo_model_path)
        
        process_video(
            input_path=input_video_path,
            output_path=processed_video_path,
            model=model,
            target_aspect_ratio=9/16,
            smoothing_factor=0.95,
            min_confidence=0.6
        )
        
        add_audio_to_video(input_video_path, processed_video_path, final_output_path)
        
        i += 1
            
        os.remove(processed_video_path)
        
def add_filter(adjustment):
    folder = "videos\\cropped"
    i = 1
    for clip in os.listdir(folder):
        filepath = os.path.join(folder, clip)
        vid = VideoFileClip(filepath)
        if adjustment == "grain":
            vid = vid.with_effects([vfx.Painting(1.5, 0.1)])
        elif adjustment =="multiply":
            vid = vid.with_effects([vfx.LumContrast(lum=90, contrast=1.3, contrast_threshold=110)])
        elif adjustment == "flip":
            vid = vid.with_effects([vfx.MirrorX()])
        else:
            os.rename(filepath, f"videos\\filtered\\clip_{i}.mp4")
            return
        
        vid.write_videofile(f"videos\\filtered\\clip_{i}.mp4", codec="libx264")
        
        i += 1

def caption_clip(caption_style):
    def format_ms(milliseconds):
        milliseconds = int(milliseconds)
        seconds, ms = divmod(milliseconds, 1000)
        minutes, seconds = divmod(seconds, 60)
        hours, minutes = divmod(minutes, 60)

        formatted = f"{hours:02}:{minutes:02}:{seconds:02},{ms:03}"
        return(formatted)
        
    def write_srt(srt_file, i, text, start, end):
        text_list = text.split()
        
        srt_file.write(f"{i}\n")
        srt_file.write(f"{format_ms(start)} --> {format_ms(end)}\n")
        
        if len(text) > 40:
            srt_file.write(' '.join(text_list[:int(len(text_list)/2)]))
            srt_file.write('\n')
            srt_file.write(' '.join(text_list[int(len(text_list)/2):]))
        else:
            srt_file.write(f"{text}")
        srt_file.write("\n\n")


    aai.settings.api_key = os.environ["AAI_TOKEN"]
    transcriber = aai.Transcriber()
    
    folder = "videos\\filtered"
    i = 1
    for clip in os.listdir(folder):
        filepath = os.path.join(folder, clip)
        print(filepath)
        transcript = transcriber.transcribe(filepath)
        
        #Format output into srt format
        sentences = transcript.get_sentences()
        j = 1
        with open("output.srt", "w") as srt_file:
            for sentence in sentences:
                sentence_list = sentence.text.split()
                if len(sentence_list) > 12:
                    half_time = (sentence.start + sentence.end) / 2
                    
                    write_srt(srt_file, j, ' '.join(sentence_list[:int(len(sentence_list)/2)]), sentence.start, half_time)
                    i+=1
                    write_srt(srt_file, j, ' '.join(sentence_list[int(len(sentence_list)/2):]), half_time, sentence.end)
                
                else:
                    write_srt(srt_file, j, sentence.text, sentence.start, sentence.end)
                
                j += 1
        
        final_output_path = f"videos\\final\\clip_{i}.mp4"
        
        if caption_style == 1:
            style = (
                "FontName=Oswald,"
                f"FontSize=15,"
                "FontWeight=1000,"
                "Italic=0,"
                "BorderStyle=1,"
                "PrimaryColour=&H00FFFFFF,"
                "OutlineColour=&H00000000,"
                "Outline=0.3,"
                "Shadow=0,"
                "Alignment=2," 
                f"MarginV=100"
            )
        elif caption_style == 2:
            style = (
                "FontName=Oswald,"
                f"FontSize=15,"
                "FontWeight=1000,"
                "Italic=1,"
                "BorderStyle=1,"
                "PrimaryColour=&H00FFFFFF,"
                "OutlineColour=&H00000000,"
                "Outline=0.3,"
                "Shadow=0,"
                "Alignment=2," 
                f"MarginV=100"
            )
        elif caption_style == 3:
            style = (
                "FontName=Oswald,"
                f"FontSize=15,"
                "FontWeight=1000,"
                "Italic=0,"
                "BorderStyle=3,"
                "PrimaryColour=&H00FFFFFF,"
                "OutlineColour=&H80000000,"
                "Outline=0.3,"
                "Shadow=0,"
                "Alignment=2," 
                f"MarginV=100"
            )
        else:
            style = (
                "FontName=Oswald,"
                f"FontSize=15,"
                "FontWeight=1000,"
                "Italic=1,"
                "BorderStyle=3,"
                "PrimaryColour=&H00FFFFFF,"
                "OutlineColour=&H80000000,"
                "Outline=0.3,"
                "Shadow=0,"
                "Alignment=2," 
                f"MarginV=100"
            )

        ffmpeg.input(filepath).output(
            final_output_path,
            vf=f"subtitles='output.srt':fontsdir=./fonts:force_style='{style}'"
        ).run()
        
        i += 1



In [7]:
folder = 'videos'
for root, dirs, files in os.walk(folder):
    for file in files:
        file_path = os.path.join(root, file)
        os.remove(file_path)

In [8]:
with open("transcript.txt", "r") as f:
    transcript = f.read()

In [9]:
clip_length = "20"
num_clips = 3
chosen_sections = ast.literal_eval(choose_sections(clip_length, num_clips, transcript))

In [10]:
print(chosen_sections)

[(160, 25120), (247560, 287360), (398090, 422300)]


In [11]:
cut_video(chosen_sections)

In [14]:
crop_clips()


0: 384x640 1 person, 66.6ms
Speed: 3.2ms preprocess, 66.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 66.9ms
Speed: 3.8ms preprocess, 66.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 61.8ms
Speed: 2.1ms preprocess, 61.8ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 113.4ms
Speed: 3.2ms preprocess, 113.4ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 91.0ms
Speed: 8.9ms preprocess, 91.0ms inference, 6.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 62.9ms
Speed: 2.8ms preprocess, 62.9ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 56.4ms
Speed: 2.4ms preprocess, 56.4ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 55.3ms
Speed: 2.6ms preprocess, 55.3ms inference, 3.3ms postprocess per image at shape (1, 3, 

In [20]:
add_filter("grain")

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2mp41', 'encoder': 'Lavf61.9.107'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1080, 1920], 'bitrate': 11920, 'fps': 29.97002997002997, 'codec_name': 'mpeg4', 'profile': '(Simple Profile)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 48000, 'bitrate': 129, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 24.99, 'bitrate': 12057, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'mpeg4', 'video_profile': '(Simple Profile)', 'video_size': [1080, 1920], 'video_bitrate': 11920, 'video_fps': 29.97

                                                                        
                                                                        
frame_index:  33%|███▎      | 241/725 [02:26<01:41,  4.77it/s, now=None]

MoviePy - Building video videos\filtered\clip_1.mp4.
MoviePy - Writing audio in clip_1TEMP_MPY_wvf_snd.mp3







                                                                        
                                                                        
frame_index:  33%|███▎      | 241/725 [02:27<01:41,  4.77it/s, now=None]

MoviePy - Done.
MoviePy - Writing video videos\filtered\clip_1.mp4



...

MoviePy - Building video videos\filtered\clip_3.mp4.
MoviePy - Writing audio in clip_3TEMP_MPY_wvf_snd.mp3







                                                                        
                                                                        
frame_index:  33%|███▎      | 241/725 [08:37<01:41,  4.77it/s, now=None]

MoviePy - Done.
MoviePy - Writing video videos\filtered\clip_3.mp4
























































































































































































































































































































































































































































































































































































































































































































































                                                                        
                                                                        
frame_index:  33%|███▎      | 241/725 [10:48<01:41,  4.77it/s, now=None]

MoviePy - Done !
MoviePy - video ready videos\filtered\clip_3.mp4


In [23]:
caption_clip(1)

videos\filtered\clip_1.mp4


RemoteProtocolError: Server disconnected without sending a response.

In [2]:
def format_ms(milliseconds):
    milliseconds = int(milliseconds)
    seconds, ms = divmod(milliseconds, 1000)
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)

    formatted = f"{hours:02}:{minutes:02}:{seconds:02},{ms:03}"
    return(formatted)
    
def write_srt(srt_file, i, text, start, end):
    text_list = text.split()
    
    srt_file.write(f"{i}\n")
    srt_file.write(f"{format_ms(start)} --> {format_ms(end)}\n")
    
    if len(text) > 40:
        srt_file.write(' '.join(text_list[:int(len(text_list)/2)]))
        srt_file.write('\n')
        srt_file.write(' '.join(text_list[int(len(text_list)/2):]))
    else:
        srt_file.write(f"{text}")
    srt_file.write("\n\n")


aai.settings.api_key = os.environ["AAI_TOKEN"]
transcriber = aai.Transcriber()

folder = "videos\\filtered"

In [9]:
caption_style = 2

i = 1
for clip in os.listdir(folder):
    filepath = os.path.join(folder, clip)
    print(filepath)
    transcript = transcriber.transcribe(filepath)

    #Format output into srt format
    sentences = transcript.get_sentences()
    print("sentences done")
    j = 1
    with open("output.srt", "w") as srt_file:
        for sentence in sentences:
            sentence_list = sentence.text.split()
            if len(sentence_list) > 12:
                half_time = (sentence.start + sentence.end) / 2
                
                write_srt(srt_file, j, ' '.join(sentence_list[:int(len(sentence_list)/2)]), sentence.start, half_time)
                i+=1
                write_srt(srt_file, j, ' '.join(sentence_list[int(len(sentence_list)/2):]), half_time, sentence.end)
            
            else:
                write_srt(srt_file, j, sentence.text, sentence.start, sentence.end)
            
            j += 1
    
    final_output_path = f"videos\\final\\clip_{i}.mp4"

    if caption_style == 1:
        style = (
            "FontName=Oswald,"
            f"FontSize=15,"
            "FontWeight=1000,"
            "Italic=0,"
            "BorderStyle=1,"
            "PrimaryColour=&H00FFFFFF,"
            "OutlineColour=&H00000000,"
            "Outline=0.3,"
            "Shadow=0,"
            "Alignment=2," 
            f"MarginV=100"
        )
    elif caption_style == 2:
        style = (
            "FontName=Oswald,"
            f"FontSize=15,"
            "FontWeight=1000,"
            "Italic=1,"
            "BorderStyle=1,"
            "PrimaryColour=&H00FFFFFF,"
            "OutlineColour=&H00000000,"
            "Outline=0.3,"
            "Shadow=0,"
            "Alignment=2," 
            f"MarginV=100"
        )
    elif caption_style == 3:
        style = (
            "FontName=Oswald,"
            f"FontSize=15,"
            "FontWeight=1000,"
            "Italic=0,"
            "BorderStyle=3,"
            "PrimaryColour=&H00FFFFFF,"
            "OutlineColour=&H80000000,"
            "Outline=0.3,"
            "Shadow=0,"
            "Alignment=2," 
            f"MarginV=100"
        )
    else:
        style = (
            "FontName=Oswald,"
            f"FontSize=15,"
            "FontWeight=1000,"
            "Italic=1,"
            "BorderStyle=3,"
            "PrimaryColour=&H00FFFFFF,"
            "OutlineColour=&H80000000,"
            "Outline=0.3,"
            "Shadow=0,"
            "Alignment=2," 
            f"MarginV=100"
        )

    print(style)
    
    ffmpeg.input(filepath).output(
        final_output_path,
        vf=f"subtitles='output.srt':fontsdir=./fonts:force_style='{style}'"
    ).run()

    i += 1



videos\filtered\clip_1.mp4
sentences done
FontName=Oswald,FontSize=15,FontWeight=1000,Italic=1,BorderStyle=1,PrimaryColour=&H00FFFFFF,OutlineColour=&H00000000,Outline=0.3,Shadow=0,Alignment=2,MarginV=100
videos\filtered\clip_2.mp4
sentences done
FontName=Oswald,FontSize=15,FontWeight=1000,Italic=1,BorderStyle=1,PrimaryColour=&H00FFFFFF,OutlineColour=&H00000000,Outline=0.3,Shadow=0,Alignment=2,MarginV=100
videos\filtered\clip_3.mp4
sentences done
FontName=Oswald,FontSize=15,FontWeight=1000,Italic=1,BorderStyle=1,PrimaryColour=&H00FFFFFF,OutlineColour=&H00000000,Outline=0.3,Shadow=0,Alignment=2,MarginV=100


In [4]:
j = 1
with open("output.srt", "w") as srt_file:
    for sentence in sentences:
        sentence_list = sentence.text.split()
        if len(sentence_list) > 12:
            half_time = (sentence.start + sentence.end) / 2
            
            write_srt(srt_file, j, ' '.join(sentence_list[:int(len(sentence_list)/2)]), sentence.start, half_time)
            i+=1
            write_srt(srt_file, j, ' '.join(sentence_list[int(len(sentence_list)/2):]), half_time, sentence.end)
        
        else:
            write_srt(srt_file, j, sentence.text, sentence.start, sentence.end)
        
        j += 1


In [5]:
final_output_path = f"videos\\final\\clip_{i}.mp4"

if caption_style == 1:
    style = (
        "FontName=Oswald,"
        f"FontSize=15,"
        "FontWeight=1000,"
        "Italic=0,"
        "BorderStyle=1,"
        "PrimaryColour=&H00FFFFFF,"
        "OutlineColour=&H00000000,"
        "Outline=0.3,"
        "Shadow=0,"
        "Alignment=2," 
        f"MarginV=100"
    )
elif caption_style == 2:
    style = (
        "FontName=Oswald,"
        f"FontSize=15,"
        "FontWeight=1000,"
        "Italic=1,"
        "BorderStyle=1,"
        "PrimaryColour=&H00FFFFFF,"
        "OutlineColour=&H00000000,"
        "Outline=0.3,"
        "Shadow=0,"
        "Alignment=2," 
        f"MarginV=100"
    )
elif caption_style == 3:
    style = (
        "FontName=Oswald,"
        f"FontSize=15,"
        "FontWeight=1000,"
        "Italic=0,"
        "BorderStyle=3,"
        "PrimaryColour=&H00FFFFFF,"
        "OutlineColour=&H80000000,"
        "Outline=0.3,"
        "Shadow=0,"
        "Alignment=2," 
        f"MarginV=100"
    )
else:
    style = (
        "FontName=Oswald,"
        f"FontSize=15,"
        "FontWeight=1000,"
        "Italic=1,"
        "BorderStyle=3,"
        "PrimaryColour=&H00FFFFFF,"
        "OutlineColour=&H80000000,"
        "Outline=0.3,"
        "Shadow=0,"
        "Alignment=2," 
        f"MarginV=100"
    )

print(style)



FontName=Oswald,FontSize=15,FontWeight=1000,Italic=1,BorderStyle=1,PrimaryColour=&H00FFFFFF,OutlineColour=&H00000000,Outline=0.3,Shadow=0,Alignment=2,MarginV=100


In [6]:
final_output_path = 'videos\\final\\clip_3.mp4'
filepath = 'videos\\filtered\\clip_1.mp4'

In [7]:
ffmpeg.input(filepath).output(
    final_output_path,
    vf=f"subtitles='output.srt':fontsdir=./fonts:force_style='{style}'"
).run()

i += 1

In [5]:
import os, ffmpeg
folder = "videos\\filtered"
style = (
            "FontName=Oswald,"
            f"FontSize=15,"
            "FontWeight=1000,"
            "Italic=1,"
            "BorderStyle=3,"
            "PrimaryColour=&H00FFFFFF,"
            "OutlineColour=&H80000000,"
            "Outline=0.3,"
            "Shadow=0,"
            "Alignment=2," 
            f"MarginV=100"
        )
for clip in os.listdir(folder):
    filepath = os.path.join(folder, clip)
final_output_path = f"videos\\final\\clip_2.mp4"
ffmpeg.input(filepath).output(
        final_output_path,
        vf=f"subtitles='output.srt':fontsdir=./fonts:force_style='{style}'"
    ).run()

(None, None)